# Experiment v5 — Chat Alignment + Code Execution Eval (Qwen)

Added new Eval prompt and temp change

temperature=0.5,(earlier 0.7)

max_new_tokens=200,  earlier (300 just for new test)

 r=8 (earlier 16)

In [1]:
!pip install unsloth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.6/66.6 kB 2.1 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 379.2/379.2 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 41.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.2/295.2 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.9/122.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.5/170.5 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/

In [2]:
import json, csv, re, torch
from datasets import load_dataset
from unsloth import FastLanguageModel
from unsloth.chat_templates import (
    get_chat_template,
    standardize_sharegpt,
    train_on_responses_only,
)
from transformers import DataCollatorForSeq2Seq
from trl import SFTTrainer, SFTConfig

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [3]:
# CONFIG
MODEL_NAME = "unsloth/Qwen2.5-Coder-14B-Instruct"
MAX_SEQ_LEN = 2048
ADAPTER_DIR = "qwen25-chat-lora"

GEN_KWARGS_CHAT = dict(
    max_new_tokens=256,
    temperature=0.5,
    top_p=0.9,
)

GEN_KWARGS_CODE = dict(
    max_new_tokens=200,
    temperature=0.2,   # low temp for determinism
)

In [4]:
# CHAT EVAL PROMPTS (LOCK THESE)
# ------------------------------------------------------------
EVAL_CHAT_PROMPTS = [
    [{"role": "user", "content": "Explain attention in transformers in simple terms."}],
    [{"role": "user", "content": "Refactor this code:\nfor i in range(10):print(i)"}],
    [{"role": "user", "content": "Explain gradient accumulation and why it is useful."}],
    [{"role": "user", "content": "Continue this conversation logically:\nUser: I keep hitting CUDA OOM errors.\nAssistant:"}],
]

In [5]:
CODE_EVAL_TASKS = [
    {
        "id": "is_prime",
        "messages": [
            {
                "role": "user",
                "content": (
                    "Write a Python function is_prime(n) that returns True if n is prime. "
                    "Handle edge cases correctly."
                )
            }
        ],
        "tests": [
            # basic cases
            ("is_prime(2)", True),
            ("is_prime(3)", True),
            ("is_prime(4)", False),

            # edge cases (MOST MODELS FAIL HERE)
            ("is_prime(1)", False),
            ("is_prime(0)", False),
            ("is_prime(-7)", False),

            # larger primes
            ("is_prime(97)", True),
            ("is_prime(99)", False),

            # type robustness (optional, but revealing)
            ("is_prime(2.0)", False),
        ],
    },
    {
        "id": "reverse_string",
        "messages": [
            {
                "role": "user",
                "content": (
                    "Write a Python function reverse_string(s) that reverses a string. "
                    "Handle edge cases safely."
                )
            }
        ],
        "tests": [
            # normal cases
            ("reverse_string('abc')", "cba"),
            ("reverse_string('racecar')", "racecar"),

            # edge cases
            ("reverse_string('')", ""),
            ("reverse_string('a')", "a"),
            ("reverse_string('Hello World')", "dlroW olleH"),

            # unicode handling (VERY IMPORTANT)
            ("reverse_string('नमस्ते')", "ेत्समन"),

            # robustness
            ("reverse_string(123)", None),
        ],
    },
]


In [6]:
# ------------------------------------------------------------
# UTILITIES
# ------------------------------------------------------------
def chat_eval(model, tokenizer, prompts, title):
    FastLanguageModel.for_inference(model)
    outputs = []

    print(f"\n===== {title} =====\n")
    for i, messages in enumerate(prompts, 1):
        inputs = tokenizer.apply_chat_template(
            messages,
            tokenize=True,
            add_generation_prompt=True,
            return_tensors="pt",
        ).to("cuda")

        with torch.no_grad():
            out = model.generate(input_ids=inputs, **GEN_KWARGS_CHAT)

        text = tokenizer.decode(out[0], skip_special_tokens=True)
        outputs.append(text)

        print(f"[PROMPT {i}]")
        print(text)
        print("-" * 80)

    return outputs


def extract_python_code(text):
    blocks = re.findall(r"```python(.*?)```", text, re.S)
    if blocks:
        return blocks[0]
    return text


def run_code_tests(code, tests):
    local_env = {}
    try:
        exec(code, {}, local_env)
    except Exception as e:
        return False, f"exec error: {e}"

    for expr, expected in tests:
        try:
            result = eval(expr, {}, local_env)
            if result != expected:
                return False, f"{expr} -> {result} (expected {expected})"
        except Exception as e:
            return False, f"{expr} error: {e}"

    return True, "all tests passed"


def run_code_eval(model, tokenizer, tasks, title):
    FastLanguageModel.for_inference(model)
    results = []

    print(f"\n===== CODE EVAL: {title} =====\n")
    for task in tasks:
        inputs = tokenizer.apply_chat_template(
            task["messages"],
            tokenize=True,
            add_generation_prompt=True,
            return_tensors="pt",
        ).to("cuda")

        with torch.no_grad():
            out = model.generate(input_ids=inputs, **GEN_KWARGS_CODE)

        text = tokenizer.decode(out[0], skip_special_tokens=True)
        code = extract_python_code(text)
        passed, message = run_code_tests(code, task["tests"])

        print(f"[{task['id']}] → {'PASS' if passed else 'FAIL'}")
        if not passed:
            print("Reason:", message)
        print("-" * 80)

        results.append({
            "task_id": task["id"],
            "passed": passed,
            "message": message,
            "code": code,
        })

    return results

In [7]:
# ------------------------------------------------------------
# LOAD BASE MODEL (BEFORE)
# ------------------------------------------------------------
base_model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_NAME,
    max_seq_length=MAX_SEQ_LEN,
    load_in_4bit=True,
    dtype=None,
)

tokenizer = get_chat_template(tokenizer, chat_template="qwen-2.5")

before_chat = chat_eval(
    base_model, tokenizer, EVAL_CHAT_PROMPTS, "BEFORE FINETUNING"
)

before_code = run_code_eval(
    base_model, tokenizer, CODE_EVAL_TASKS, "BEFORE FINETUNING"
)


==((====))==  Unsloth 2026.1.1: Fast Qwen2 patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.



===== BEFORE FINETUNING =====

[PROMPT 1]
system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.
user
Explain attention in transformers in simple terms.
assistant
Attention in transformers is a mechanism that allows the model to focus on different parts of the input when making predictions. Imagine you're reading a sentence, and you want to understand what a specific word means. The attention mechanism helps the model pay more attention to the words around the word of interest, as these words provide context.

In a transformer, attention allows each position in the input sequence to weigh the importance of other positions differently. This is particularly useful for tasks like machine translation or text summarization, where understanding the relationship between different parts of the text is crucial. The attention mechanism computes a score for each pair of positions, indicating how relevant one is to the other, and then uses these scores to compute a weighted s

In [8]:
# ------------------------------------------------------------
# ADD LORA
# ------------------------------------------------------------
model = FastLanguageModel.get_peft_model(
    base_model,
    r=8,
    lora_alpha=16,
    target_modules=[
        "q_proj","k_proj","v_proj","o_proj",
        "gate_proj","up_proj","down_proj"
    ],
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
)

Unsloth 2026.1.1 patched 48 layers with 48 QKV layers, 48 O layers and 48 MLP layers.


In [9]:
# ------------------------------------------------------------
# DATASET (CHAT)
# ------------------------------------------------------------
dataset = load_dataset("mlabonne/FineTome-100k", split="train")
dataset = standardize_sharegpt(dataset)

def format_chat(examples):
    texts = [
        tokenizer.apply_chat_template(c, tokenize=False)
        for c in examples["conversations"]
    ]
    return {"text": texts}

dataset = dataset.map(format_chat, batched=True)


README.md:   0%|          | 0.00/982 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/117M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/100000 [00:00<?, ? examples/s]

Unsloth: Standardizing formats (num_proc=2):   0%|          | 0/100000 [00:00<?, ? examples/s]

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

In [10]:

# ------------------------------------------------------------
# TRAIN
# ------------------------------------------------------------
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    data_collator=DataCollatorForSeq2Seq(tokenizer),
    max_seq_length=MAX_SEQ_LEN,
    packing=False,
    args=SFTConfig(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        max_steps=30,              # demo run
        learning_rate=2e-4,
        logging_steps=1,
        optim="paged_adamw_8bit",
        output_dir="outputs",
        report_to="none",
    ),
)

trainer = train_on_responses_only(
    trainer,
    instruction_part="<|im_start|>user\n",
    response_part="<|im_start|>assistant\n",
)

trainer.train()

model.save_pretrained(ADAPTER_DIR)
tokenizer.save_pretrained(ADAPTER_DIR)

Unsloth: Tokenizing ["text"] (num_proc=4):   0%|          | 0/100000 [00:00<?, ? examples/s]

Map (num_proc=6):   0%|          | 0/100000 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 100,000 | Num Epochs = 1 | Total steps = 30
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 4 x 1) = 4
 "-____-"     Trainable parameters = 68,812,800 of 14,838,846,464 (0.46% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,0.618000
2,0.596300
3,0.630200
4,0.549200
5,1.024900
6,0.779800
7,0.613900
8,0.717600
9,0.703300
10,0.465300


('qwen25-chat-lora/tokenizer_config.json',
 'qwen25-chat-lora/special_tokens_map.json',
 'qwen25-chat-lora/chat_template.jinja',
 'qwen25-chat-lora/vocab.json',
 'qwen25-chat-lora/merges.txt',
 'qwen25-chat-lora/added_tokens.json',
 'qwen25-chat-lora/tokenizer.json')

In [11]:
# ------------------------------------------------------------
# AFTER EVAL
# ------------------------------------------------------------
# Reuse the 'model' object that has already been finetuned with LoRA adapters.
# No need to reload the base model and then its adapter, as 'model' already contains the finetuned state.

after_chat = chat_eval(
    model, tokenizer, EVAL_CHAT_PROMPTS, "AFTER FINETUNING"
)

after_code = run_code_eval(
    model, tokenizer, CODE_EVAL_TASKS, "AFTER FINETUNING"
)


===== AFTER FINETUNING =====

[PROMPT 1]
system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.
user
Explain attention in transformers in simple terms.
assistant
Attention is a mechanism used in transformers to focus on different parts of the input data when making predictions. It allows the model to weigh the importance of different words or tokens in a sentence, giving more weight to relevant information and less weight to irrelevant information. This helps the model to better understand the context and relationships between words, which is especially useful for tasks like language translation, text summarization, and question answering.
--------------------------------------------------------------------------------
[PROMPT 2]
system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.
user
Refactor this code:
for i in range(10):print(i)
assistant
Certainly! Here's the refactored code:

```python
for i in range(10):
    print(i)
```

In this ref

In [12]:
# ------------------------------------------------------------
# EXPORT REPORTS
# ------------------------------------------------------------
chat_rows = []
for i, (b, a) in enumerate(zip(before_chat, after_chat)):
    chat_rows.append({
        "prompt_id": i,
        "before_len": len(b.split()),
        "after_len": len(a.split()),
        "before": b,
        "after": a,
    })

